# Jane Street: Neural Network Starter

I try implementing a simple Tensorflow Keras neural network here. Train in Version 17.

**Caution:** The GroupCV method applied in this notebook may cause time leakage problem. Please use [Purged Time-Series CV][1] instead.

[1]: https://www.kaggle.com/marketneutral/purged-time-series-cv-xgboost-optuna

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import os, gc
# import cudf
import pandas as pd
import numpy as np
# import cupy as cp
import janestreet
import xgboost as xgb
from hyperopt import hp, fmin, tpe, Trials
from hyperopt.pyll.base import scope
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import GroupKFold
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from joblib import dump, load

import tensorflow as tf
tf.random.set_seed(42)
import tensorflow.keras.backend as K
import tensorflow.keras.layers as layers
from tensorflow.keras.callbacks import Callback, ReduceLROnPlateau, ModelCheckpoint, EarlyStopping

# Preprocessing

In [ ]:
# print('Loading...')
# train = cudf.read_csv('/kaggle/input/jane-street-market-prediction/train.csv')
test_df = pd.read_csv('/kaggle/input/jane-street-market-prediction/example_test.csv', nrows = 3)
features = [c for c in test_df.columns if 'feature' in c]

# print('Filling...')
# f_mean = train[features[1:]].mean()
# train = train.query('weight > 0').reset_index(drop = True)
# train[features[1:]] = train[features[1:]].fillna(f_mean)
# train['action'] = (train['resp'] > 0).astype('int')

# print('Converting...')
# train = train.to_pandas()
# f_mean = f_mean.values.get()
# np.save('f_mean.npy', f_mean)

# print('Finish.')

# Training

In [ ]:
def create_mlp(num_columns, num_labels, hidden_units, dropout_rates, label_smoothing, learning_rate):
    
    inp = tf.keras.layers.Input(shape = (num_columns, ))
    x = tf.keras.layers.BatchNormalization()(inp)
    x = tf.keras.layers.Dropout(dropout_rates[0])(x)
    for i in range(len(hidden_units)): 
        x = tf.keras.layers.Dense(hidden_units[i])(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Activation(tf.keras.activations.swish)(x)
        x = tf.keras.layers.Dropout(dropout_rates[i+1])(x)    
        
    x = tf.keras.layers.Dense(num_labels)(x)
    out = tf.keras.layers.Activation('sigmoid')(x)
    
    model = tf.keras.models.Model(inputs = inp, outputs = out)
    model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate),
                  loss = tf.keras.losses.BinaryCrossentropy(label_smoothing = label_smoothing), 
                  metrics = tf.keras.metrics.AUC(name = 'AUC'), 
                 )
    
    return model

price_index = list(range(1, 41))
buy_index = list(range(84, 96))
sell_index = list(range(96, 108))

def FE_infer(test_data):
    test_data[0, buy_index+sell_index] = test_data[0, sell_index+buy_index]
    test_data[0, price_index] = - test_data[0, price_index]
    return test_data

In [ ]:
batch_size = 4096
hidden_units = [160, 160, 160]
dropout_rates = [0.2, 0.2, 0.2, 0.2]
label_smoothing = 1e-2
learning_rate = 1e-3

# oof = np.zeros(len(train['action']))
# gkf = GroupKFold(n_splits = 5)
# for fold, (tr, te) in enumerate(gkf.split(train['action'].values, train['action'].values, train['date'].values)):
    
#     X_tr, X_val = train.loc[tr, features].values, train.loc[te, features].values
#     y_tr, y_val = train.loc[tr, 'action'].values, train.loc[te, 'action'].values
    
#     ckp_path = f'JSModel_{fold}.hdf5'
#     model = create_mlp(X_tr.shape[1], 1, hidden_units, dropout_rates, label_smoothing, learning_rate)
#     rlr = ReduceLROnPlateau(monitor = 'val_AUC', factor = 0.1, patience = 3, verbose = 0, 
#                             min_delta = 1e-4, mode = 'max')
#     ckp = ModelCheckpoint(ckp_path, monitor = 'val_AUC', verbose = 0, 
#                           save_best_only = True, save_weights_only = True, mode = 'max')
#     es = EarlyStopping(monitor = 'val_AUC', min_delta = 1e-4, patience = 7, mode = 'max', 
#                        baseline = None, restore_best_weights = True, verbose = 0)
#     model.fit(X_tr, y_tr, validation_data = (X_val, y_val), epochs = 1000, 
#               batch_size = batch_size, callbacks = [rlr, ckp, es], verbose = 0)
                
#     oof[te] += model.predict(X_val, batch_size = batch_size * 4).ravel()
#     score = roc_auc_score(y_val, oof[te])
#     print(f'Fold {fold} ROC AUC:\t', score)
    
#     # Finetune 3 epochs on validation set with small learning rate
#     model = create_mlp(X_tr.shape[1], 1, hidden_units, dropout_rates, label_smoothing, learning_rate / 100)
#     model.load_weights(ckp_path)
#     model.fit(X_val, y_val, epochs = 3, batch_size = batch_size, verbose = 0)
#     model.save_weights(ckp_path)
    
#     K.clear_session()
#     del model
#     rubbish = gc.collect()

In [ ]:
# score_oof = roc_auc_score(train['action'].values, oof)
# print(score_oof)

# Load Models

In [ ]:
num_models = 3

models = []
for i in range(num_models):
    clf = create_mlp(len(features), 5, hidden_units, dropout_rates, label_smoothing, learning_rate)
    clf.load_weights(f'../input/nn-ensemble-all-normal-busy-fe-by-side/JSNN_model_{i}.h5')

    models.append(clf)

In [ ]:
f_mean = np.load('../input/na-filling/f_mean.npy')
f_median = np.load('../input/na-filling/f_median.npy')

# Submitting

Just use two models to reduce running time.

In [ ]:
env = janestreet.make_env()
env_iter = env.iter_test()

In [ ]:
th = 0.50
ensemble_weight = [0.3, 0.5, 0.2]
index_features = [n for n,col in enumerate(test_df.columns) if col in features]

for (test_df, pred_df) in tqdm(env.iter_test()):
    if test_df['weight'].item() > 0:
        # FE:
        x_tt = test_df.values[0, index_features].reshape(1,-1)
        if np.isnan(x_tt[0, :].sum()):
            x_tt[:, :] = np.nan_to_num(x_tt[:, :]) + np.isnan(x_tt[:, :]) * f_median
        if test_df['feature_0'].item()<0:
            x_tt = FE_infer(x_tt)
            
        # Inference:
        pred = np.median(np.average([model(x_tt, training=False).numpy() for model in models], axis=0, weights=ensemble_weight))
        pred_df["action"].values[0] = np.where(pred >= th, 1, 0).astype(int)
    else:
        pred_df["action"].values[0] = 0
    env.predict(pred_df)